In [1]:
import torch
import random
from PIL import Image
import os
from torchvision.datasets import CIFAR10, CIFAR100, CocoCaptions, ImageNet
import slip_models
from tokenizer import SimpleTokenizer

import torchvision.transforms as T
from torchvision import datasets, transforms


from tqdm import tqdm
import json
import matplotlib.pyplot as plt

import numpy as np
%pip install ipywidgets
%pip install update tqdm%matplotlib inline


You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
ERROR: Invalid requirement: 'tqdm%matplotlib'
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
!/export/home/google-cloud-sdk/bin/gsutil -m cp gs://sfr-tpu-us-east1-research/bwallace/ckpts/models/multimodel_clip/parallel_vision_standard_text/viz_parallel_text_standard_bs_1024_epoch_35.ckpt ckpts/viz_parallel_text_standard_bs_1024_epoch_35.ckpt
    



Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update

Copying gs://sfr-tpu-us-east1-research/bwallace/ckpts/models/multimodel_clip/parallel_vision_standard_text/viz_parallel_text_standard_bs_1024_epoch_35.ckpt...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

/ [1/1 files][  1.4 GiB/  1.4 GiB] 100% Done                                    
Operation completed over 1 objects/1.4 GiB.                                      


In [3]:
preprocess = T.Compose(
                    [
                        T.Resize(256),
                        T.CenterCrop(224),
                        T.ToTensor(),
                        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                    ]
                        )

grayscale_preprocess = T.Compose(
                    [
                        T.Resize(256),
                        T.CenterCrop(224),
                        T.ToTensor(),
                        T.Normalize(mean=[np.average([0.485, 0.456, 0.406])], std=[np.average([0.229, 0.224, 0.225])]),
                        T.Lambda(lambda x: x.repeat(3, 1, 1) ),
                    ]
                        )

In [4]:
grayscale_preprocess

Compose(
    Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.449], std=[0.226])
    Lambda()
)

In [85]:
# model = slip_models.CLIP_VITB16()
# ckpt = torch.load('ckpts/clip_vit_epoch_35.ckpt', map_location='cpu')

model = slip_models.VisionParallelTextStandard(64, 8)
ckpt = torch.load('ckpts/viz_parallel_text_standard_bs_1024_epoch_35.ckpt', map_location='cpu')
# ckpt = torch.load('tmpp_save/test_epoch_4.ckpt', map_location='cpu')


# model = slip_models.CLIP_VITB16(num_prompt_tokens=64, num_text_outputs=1000)
# ckpt = torch.load('ckpts/epoch_30_prompted_clip_may_27.ckpt', map_location='cpu')


# model.load_state_dict({k.replace('module.',''):v for k,v in ckpt["model"].items()})
model.load_state_dict(ckpt["model"])

model.eval()
model = model.cuda()

In [6]:
cc_model = slip_models.CLIP_VITB16(ssl_mlp_dim=4096, ssl_emb_dim=256)


ckpt = torch.load('ckpts/clip_base_cc12m_35ep.pt', map_location='cpu')
# ckpt = torch.load('ckpts/slip_base_100ep.pt', map_location='cpu')
cc_model.load_state_dict({k.replace('module.',''):v for k,v in ckpt["state_dict"].items()})
cc_model = cc_model.cuda()

In [7]:
tokenizer = SimpleTokenizer()

In [23]:
# image = preprocess(Image.open("pics/golden-retriever.png")).unsqueeze(0).to(device)
# image = preprocess(Image.open("pics/CLIP.png")).unsqueeze(0).to(device)
base_text = ["a diagram", "a dog", "a cat"]
model.eval()
dog_image = preprocess(Image.open("pics/golden-retriever.png"))
diagram_image = preprocess(Image.open("pics/CLIP.png").convert("RGB"))
cat_image = preprocess(Image.open("pics/cat.jpg"))
images = torch.stack([diagram_image, dog_image, cat_image]).cuda()

text = tokenizer([f"a picture of {s}" for s in base_text]).cuda()

with torch.no_grad():
    image_features = model.encode_image(images)
    text_features = model.encode_text(text)
    
    # image_features = image_features / image_features.norm(dim=1, keepdim=True)
    # text_features = text_features / text_features.norm(dim=1, keepdim=True)
    
    
    logits_per_image = model.logit_scale.exp() * image_features @ text_features.t()
    # logits_per_image, logits_per_text = model(images, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

Label probs: [[0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [9]:
# image = preprocess(Image.open("pics/golden-retriever.png")).unsqueeze(0).to(device)
# image = preprocess(Image.open("pics/CLIP.png")).unsqueeze(0).to(device)
base_text = ["a diagram", "a dog", "a cat"]
model.eval()
dog_image = preprocess(Image.open("pics/golden-retriever.png"))
diagram_image = preprocess(Image.open("pics/CLIP.png").convert("RGB"))
cat_image = preprocess(Image.open("pics/cat.jpg"))
images = torch.stack([diagram_image, dog_image, cat_image]).cuda()

text = tokenizer([f"a picture of {s}" for s in base_text]).cuda()

with torch.no_grad():
    image_features = cc_model.encode_image(images)
    text_features = cc_model.encode_text(text)
    
    # image_features = image_features / image_features.norm(dim=1, keepdim=True)
    # text_features = text_features / text_features.norm(dim=1, keepdim=True)
    
    
    logits_per_image = model.logit_scale.exp() * image_features @ text_features.t()
    # logits_per_image, logits_per_text = model(images, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

Label probs: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


# Basic ImageNet and CIFAR checks

In [24]:
# Download the dataset
cifar10 = CIFAR10(root="/tmp/", transform=preprocess, download=True, train=False)
loader = torch.utils.data.DataLoader(cifar10, batch_size=32, shuffle=True)
# text_inputs = torch.stack([tokenizer(f"a photo of a {c}") for c in cifar10.classes]).cuda()


text_inputs = torch.stack([tokenizer(f"a photo of a {c}") for c in cifar10.classes]).cuda()


num_correct = 0
num_seen = 0
with torch.no_grad():
    #  text_features = model.encode_text(text_inputs)
    for imgs, targets in loader:
        imgs = imgs.cuda()
        targets = targets.cuda()
        
        # image_features = model.encode_image(imgs)
        # results = model(imgs, text_inputs, lang_prompt_viz=True, sharded_computation=False)
        # results = cc_model(imgs, text_inputs)
        image_features = model.encode_image(imgs)
        text_features = model.encode_text(text_inputs)
        
        image_features = image_features / image_features.norm(dim=1, keepdim=True)
        text_features = text_features / text_features.norm(dim=1, keepdim=True)
        # image_features = (image_features - image_features.mean(0)) / torch.sqrt(image_features.var(0))
        # print("Using barlow"); text_features = (text_features - text_features.mean(0)) / torch.sqrt(text_features.var(0)) 
        # image_features = (image_features - coco_image_mean) / torch.sqrt(coco_image_var)
        # print("Using barlow"); text_features = (text_features - coco_text_mean) / torch.sqrt(coco_text_var)  
        
    
        similarity = (image_features @ text_features.T).softmax(dim=-1)
        max_sims = similarity.max(dim=-1)[0]
        # print(max_sims.min(), max_sims.max(), max_sims.mean())
        num_correct += (similarity.argmax(dim=-1)==targets).sum().item()  
        num_seen += imgs.shape[0]
        curr_acc = num_correct / num_seen
        print(f"{num_seen} / {len(cifar10)} // Current Acc: {curr_acc}")
        if num_seen > 1000: break
acc = num_correct / num_seen
print(f"(PARTIAL) Final Acc {acc}")

Files already downloaded and verified
32 / 10000 // Current Acc: 0.90625
64 / 10000 // Current Acc: 0.78125
96 / 10000 // Current Acc: 0.78125
128 / 10000 // Current Acc: 0.7578125
160 / 10000 // Current Acc: 0.73125
192 / 10000 // Current Acc: 0.7083333333333334
224 / 10000 // Current Acc: 0.6964285714285714
256 / 10000 // Current Acc: 0.6953125
288 / 10000 // Current Acc: 0.7013888888888888
320 / 10000 // Current Acc: 0.690625
352 / 10000 // Current Acc: 0.6846590909090909
384 / 10000 // Current Acc: 0.6796875
416 / 10000 // Current Acc: 0.6634615384615384
448 / 10000 // Current Acc: 0.6674107142857143
480 / 10000 // Current Acc: 0.6666666666666666
512 / 10000 // Current Acc: 0.67578125
544 / 10000 // Current Acc: 0.6746323529411765
576 / 10000 // Current Acc: 0.6753472222222222
608 / 10000 // Current Acc: 0.6825657894736842
640 / 10000 // Current Acc: 0.6875
672 / 10000 // Current Acc: 0.6875
704 / 10000 // Current Acc: 0.6889204545454546
736 / 10000 // Current Acc: 0.69021739130434

In [13]:
imagenet = ImageNet(root="/export/share/datasets/vision/imagenet", transform=preprocess, split='val')
loader = torch.utils.data.DataLoader(imagenet, batch_size=32, num_workers=4, shuffle=True)
text_inputs = torch.stack([tokenizer(f"a photo of a {c}") for c in imagenet.classes]).cuda()

num_correct = 0
num_seen = 0
with torch.no_grad():
    text_features = model.encode_text(text_inputs)
    text_features = text_features / text_features.norm(dim=1, keepdim=True)
    # print("Using barlow"); text_features = (text_features - text_features.mean(0)) / torch.sqrt(text_features.var(0))
    # print("Using barlow"); text_features = (text_features - coco_text_mean) / torch.sqrt(coco_text_var)  
    
    for imgs, targets in loader:
        imgs = imgs.cuda()
        targets = targets.cuda()
        
        image_features = model.encode_image(imgs)
        image_features = image_features / image_features.norm(dim=1, keepdim=True)
        # print("using barlow"); image_features = (image_features - image_features.mean(0)) / torch.sqrt(image_features.var(0))
        # results = model(imgs, text_inputs, lang_prompt_viz=True, sharded_computation=False)
        # image_features = results['image_embed']
        # text_features = results['text_embed']
        # image_features = (image_features - coco_image_mean) / torch.sqrt(coco_image_var) 
        
        
        similarity = (image_features @ text_features.T).softmax(dim=-1)
        num_correct += (similarity.argmax(dim=-1)==targets).sum().item()  
        max_sims = similarity.max(dim=-1)[0]
        # print(max_sims.min(), max_sims.max(), max_sims.mean())
        num_seen += imgs.shape[0]
        curr_acc = num_correct / num_seen
        print(f"{num_seen} / {len(imagenet)} // Current Acc: {curr_acc}")
        if num_seen > 1000: break
        
print(f"(PARTIAL) Final Acc {curr_acc}")

32 / 50000 // Current Acc: 0.0
64 / 50000 // Current Acc: 0.0625
96 / 50000 // Current Acc: 0.052083333333333336
128 / 50000 // Current Acc: 0.078125
160 / 50000 // Current Acc: 0.09375
192 / 50000 // Current Acc: 0.10416666666666667
224 / 50000 // Current Acc: 0.11607142857142858
256 / 50000 // Current Acc: 0.11328125
288 / 50000 // Current Acc: 0.1076388888888889
320 / 50000 // Current Acc: 0.1125
352 / 50000 // Current Acc: 0.10795454545454546
384 / 50000 // Current Acc: 0.10677083333333333
416 / 50000 // Current Acc: 0.11057692307692307
448 / 50000 // Current Acc: 0.109375
480 / 50000 // Current Acc: 0.11458333333333333
512 / 50000 // Current Acc: 0.111328125
544 / 50000 // Current Acc: 0.10845588235294118
576 / 50000 // Current Acc: 0.109375
608 / 50000 // Current Acc: 0.1118421052631579
640 / 50000 // Current Acc: 0.1109375
672 / 50000 // Current Acc: 0.11160714285714286
704 / 50000 // Current Acc: 0.11079545454545454
736 / 50000 // Current Acc: 0.11005434782608696
768 / 50000 //

In [14]:


def dataset_constructor(name, split, transform):
    """
    inputs
        name: string that we'll run if/else on
        split: "train" or "val" for now
    """
    assert split in ['train', 'val']

    # Need to figure out way to override the getitem call for strong weak augmentatin
    if name == 'cifar10':
        # Has class names
        return datasets.CIFAR10(root=os.path.expanduser("~/.cache"),
                                download=True,
                                train=(split=='train'),
                               transform=transform)
    elif name == 'cifar100':
        # Has class names
        return datasets.CIFAR100(root=os.path.expanduser("~/.cache"),
                                download=True,
                                train=(split=='train'),
                                transform=transform)

    elif name == 'svhn':
        # Doesn't have class names
        return datasets.SVHN(root=os.path.expanduser("~/.cache"),
                             download=True,
                             transform=transform,
                             split=split)
    elif name == 'gtsrb': #####
        assert split=='val'
        return datasets.ImageFolder(root="/export/share/bwallace/datasets/gtsrb/pytorch_format",
                                   transform=transform)
        return datasets.GTSRB(root=os.path.expanduser("~/.cache"), split='test', download=True)
    elif name == 'food101':
        # name is in .classes
        assert split=='val'
        return datasets.ImageFolder(root="/export/share/bwallace/datasets/food101/images/",
                   transform=transform)
    elif name == 'merced':
        # name is in .classes
        assert split=='val'
        return datasets.ImageFolder(root="/export/share/bwallace/datasets/UCMerced_LandUse/Images/",
                   transform=transform)
    elif name == 'mnist':
        assert split=='val'
        return datasets.MNIST(root=os.path.expanduser("~/.cache"), train=False, download=True, transform=transform)
    elif name == 'flowers': ####
        assert split=='val'
        print("NOTE PROPER METRIC IS MEAN PER CLASS")
        return datasets.ImageFolder(root="/export/share/bwallace/datasets/flowers/test/",
                                    transform=transform)
    elif name == 'aircraft': ####
        assert split=='val'
        print("NOTE PROPER METRIC IS MEAN PER CLASS")
        return datasets.ImageFolder(root="/export/share/bwallace/datasets/aircraft/data/test_pytorch_format/",
                                   transform=transform)
    elif name == 'cars':
        assert split=='val'
        return datasets.ImageFolder(root="/export/share/datasets/vision/stanford_cars/car_data/test/",
                                   transform=transform)
    elif name == 'eurosat':
        assert split=='val'
        return datasets.ImageFolder(root="/export/share/datasets/vision/euro_sat/2750/",
                                   transform=transform)
    elif name == 'dtd':
        # name is in .classes
        return datasets.ImageFolder(root=f"/export/share/bwallace/datasets/dtd/{split}",
                   transform=transform)
    elif name == 'cub':
        # name is in .classes
        return datasets.ImageFolder(root=f"/export/share/bwallace/datasets/CUB_2011_formatted/{split}",
                   transform=transform)
    elif name == 'places365':
        # class names works in 
        return datasets.Places365(
                    root="/export/share/datasets/vision/Places365/",
                    split='train-standard' if split=='train' else split,
                   transform=transform)

    elif name == 'imagenet':
        # Doesn't have class names, have separate call
        return datasets.ImageNet(
                    root="/export/share/datasets/vision/imagenet/",
                    split=split,
                   transform=transform)
    elif name == 'imagenet_val':
        assert split=='train'
        # Doesn't have class names, only use is having Imagenet val be trainset 
        return datasets.ImageNet(
                    root="/export/share/datasets/vision/imagenet/",
                    split='val',
                   transform=transform)
    else:
        raise NotImplementedError


def get_imagenet_class_dict():
        idx_to_word_id_and_name_tuple = json.load(open('imagenet_class_index.json'))
        word_id_to_name_and_idx = {v[0]:(v[1],int(k))
                           for k,v in idx_to_word_id_and_name_tuple.items()}
        # e.g. n0023923939 to ('unicorn', 123)
        return word_id_to_name_and_idx

In [79]:
def test_on_dataset(model, dataset_name, transform,
                   prompt_template=None,
                   test_components_mode='standard',
                   normalization_from_all_components=False,
                   component_grouping=1):
    """
    Options for test components mode
        standard : standard
        test_all : scale through
        random_all : As all but random sorting
        
    Normalization is whether to normalize from all components or just ones in computation
    
    """
    dataset = dataset_constructor(dataset_name, 'val', transform)
    loader = torch.utils.data.DataLoader(dataset, batch_size=32, num_workers=4)
    
    with torch.no_grad():
        
        
        if prompt_template is not None:
            class_names = [prompt_template.format(c.replace("'","")) for c in dataset.classes]
            # print(class_names)
            text_inputs = torch.stack([tokenizer(txt) for txt in class_names]).cuda()
            text_features = model.encode_text(text_inputs)
            text_features = text_features / text_features.norm(dim=1, keepdim=True)
        else:
            with open('templates.json') as f:
                all_templates = json.load(f)
            templates = all_templates[dataset_name]
            text_features = []
            class_names = [c.replace("'", "") for c in dataset.classes]
            for class_name in class_names:
                text_inputs = torch.stack([tokenizer(t.format(class_name)) for t in templates]).cuda()
                class_features = model.encode_text(text_inputs)
                class_features = class_features / class_features.norm(dim=-1, keepdim=True)
                # print(class_features.shape)
                class_features = class_features.mean(dim=0)
                class_features = class_features / class_features.norm(dim=-1, keepdim=False)
                # print(class_features.shape)
                text_features.append(class_features)
            text_features = torch.stack(text_features)
            # print(text_features.shape)
                
        if test_components_mode in ['standard']:
            sorted_feature_variances, sorted_variance_idx = None, None
        elif test_components_mode in ['test_all', 'ensemble']:
            # grouping will return indices in blocks (idx //component_grouping will be repeated numbers in blocks of component_grouping)
            sorted_feature_variances, sorted_variance_idx = sorted_feature_component_variances(text_features,
                                                                                              group_size=component_grouping)
        elif test_components_mode == 'random_all':
            # Want random to be grouped as well
            sorted_feature_variances, sorted_variance_idx = sorted_feature_component_variances(text_features,
                                                                                              group_size=component_grouping)
            grouped_idx = torch.randperm(sorted_variance_idx.shape[0] // component_grouping)
            sorted_variance_idx = torch.cat([torch.arange(component_grouping).cuda() + (component_grouping*i) for i in grouped_idx])
            # print(sorted_variance_idx)
        else:
            raise NotImplementedError
        
        num_correct = 0 if test_components_mode =='standard' else torch.zeros(text_features.shape[1])
        num_seen = 0
        for imgs, targets in tqdm(loader):
            imgs = imgs.cuda()
            targets = targets.cuda()

            image_features = model.encode_image(imgs)
            image_features = image_features / image_features.norm(dim=1, keepdim=True)
            # results = model(imgs, text_inputs, lang_prompt_viz=True, sharded_computation=False)
            # image_features = results['image_embed']
            # text_features = results['text_embed']


            if test_components_mode=="standard":
                similarity = (image_features @ text_features.T).softmax(dim=-1)
                num_correct += (similarity.argmax(dim=-1)==targets).sum().item() 
            elif test_components_mode in ["test_all", "random_all", "ensemble"]:
                for num_components in range(component_grouping, text_features.shape[1]+1,
                                           component_grouping):
                    idx_to_use = sorted_variance_idx[:num_components]
                    masked_text_features = text_features[:, idx_to_use]
                    masked_image_features = image_features[:, idx_to_use]
                    if not normalization_from_all_components:
                        masked_text_features =  masked_text_features / masked_text_features.norm(dim=1, keepdim=True)
                        masked_image_features =  masked_image_features / masked_image_features.norm(dim=1, keepdim=True)
                    if test_components_mode in ["test_all", "random_all"]:
                        masked_similarity = (masked_image_features @ masked_text_features.T).softmax(dim=-1)
                        num_correct[num_components-1] += (masked_similarity.argmax(dim=-1)==targets).sum().item()
                    elif test_components_mode == 'ensemble':
                        # right now features are bs x (head_dim * num_components)
                        # put as bs x num_components x head_dim
                        # permute to num_components x bs x head_dim
                        # BMM against num_components x head_dim x bs
                        # get num_components bs x bs selection matrices
                        # print(masked_image_features.shape)
                        # print(masked_text_features.shape)
                        num_texts = masked_text_features.shape[0]
                        num_images = masked_image_features.shape[0]
                        masked_image_features = masked_image_features.view(num_images, -1, component_grouping).permute(1, 0, 2)
                        masked_text_features = masked_text_features.view(num_texts, -1, component_grouping).permute(1, 2, 0)
                        per_component_sims = torch.bmm(masked_image_features, masked_text_features) # num_components x num_images x num_texts
                        per_image_per_component_preds = per_component_sims.argmax(dim=-1) # num_components x num_images
                        # print(per_image_per_component_preds)
                        # print(per_image_per_component_preds.shape)
                        ensemble_pred = per_image_per_component_preds.cpu().mode(0)[0]
                        # print(ensemble_pred.shape, targets.shape)
                        num_correct[num_components-1] += (ensemble_pred == targets.cpu()).sum().item()
                        # BMM against bs x head_dim x num_components and get 
                        # bs x num_components x num_components
            else:
                 raise NotImplementedError
            # max_sims = similarity.max(dim=-1)[0]
            # print(max_sims.min(), max_sims.max(), max_sims.mean())
            num_seen += imgs.shape[0]
            curr_acc = num_correct / num_seen
    return curr_acc


def get_dataset_text_features(model, dataset_name, norm=True, print_class_names=False):
    dataset = dataset_constructor(dataset_name, 'val', None)
    class_names = [f"a photo of a {c}" for c in dataset.classes]
    if print_class_names:
            print(dataset.classes)
    text_inputs = torch.stack([tokenizer(txt) for txt in class_names]).cuda()
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)
        if norm: text_features = text_features / text_features.norm(dim=1, keepdim=True)
    return text_features

def sorted_feature_component_variances(tensor, group_size=1):
    # NOTE: sorting max first so [0] is intuitively important
    feature_variances = tensor.var(dim=0)
    grouped_feature_variances = feature_variances.view(-1, group_size).mean(1)
    grouped_idx = grouped_feature_variances.argsort().flip(0)
    # will be # components / group_size
    idx = torch.cat([torch.arange(group_size).cuda() + (group_size*i) for i in grouped_idx])
    return feature_variances[idx], idx


In [80]:
def plot_ordered_vs_random_acc_samples(ordered_accs, random_acc_sampling,
                                       baseline_ordered_accs=None, baseline_random_acc_sampling=None,
                                       title=None):
    random_accs = torch.mean(torch.stack(random_acc_sampling, dim=0), dim=0)
    plt.figure()
    plt.title(title)
    plt.ylabel('accuracy')
    plt.xlabel('# components used')
    plt.plot(ordered_accs.detach().cpu().numpy(), label="Sorted Variances")
    plt.plot(random_accs.detach().cpu().numpy(), label=f"Random (average {len(random_acc_sampling)} runs)")
    if baseline_ordered_accs is not None:
        plt.plot(baseline_ordered_accs.detach().cpu().numpy(), label="Baseline sorted")
    if baseline_random_acc_sampling is not None:
        plt.plot(torch.stack(baseline_random_acc_sampling).mean(0).detach().cpu().numpy(), label=f"Baseline Random (average {len(baseline_random_acc_sampling)} runs)")
    plt.legend()
    plt.show()
    
    
def generate_order_variance_plot(model, ds_name, preprocess, num_samples=2, group_size=1, global_norm=False,
                                    baseline_model=None):
    random_acc_sampling = [test_on_dataset(model, ds_name, preprocess,
                                           test_components_mode='random_all',
                                          component_grouping=group_size,
                                          normalization_from_all_components=global_norm) for _ in range(num_samples)]
    ordered_accs = test_on_dataset(model, ds_name, preprocess,
                                   test_components_mode='test_all', component_grouping=group_size,
                                          normalization_from_all_components=global_norm)
    
    baseline_random_acc_sampling = None
    baseline_ordered_accs = None
    if baseline_model is not None:
        baseline_random_acc_sampling = [test_on_dataset(baseline_model, ds_name, preprocess,
                                               test_components_mode='random_all',
                                              component_grouping=group_size,
                                              normalization_from_all_components=global_norm) for _ in range(num_samples)]
        baseline_ordered_accs = test_on_dataset(baseline_model, ds_name, preprocess,
                                       test_components_mode='test_all', component_grouping=group_size,
                                              normalization_from_all_components=global_norm)
    plot_ordered_vs_random_acc_samples(ordered_accs, random_acc_sampling, title=ds_name,
                                      baseline_ordered_accs=baseline_ordered_accs,
                                      baseline_random_acc_sampling=baseline_random_acc_sampling)


In [86]:
test_on_dataset(model, 'cifar10', preprocess, test_components_mode='ensemble',
               component_grouping=8)


Files already downloaded and verified


100%|█████████████████████████████████████████████████| 313/313 [02:36<00:00,  1.99it/s]


tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1084, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1084, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1297, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.1495, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.1159, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.1351, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.1321, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.1467, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1601,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1683, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1564, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1711, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.1829, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.1856, 

In [87]:
test_on_dataset(model, 'cifar10', preprocess, test_components_mode='standard',
               component_grouping=8)


Files already downloaded and verified


100%|█████████████████████████████████████████████████| 313/313 [01:35<00:00,  3.27it/s]


0.2301

In [25]:
generate_order_variance_plot(model, 'dtd', preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

 47%|██████████████████████████████████████████████████████████████████▉                                                                          | 28/59 [00:06<00:07,  4.29it/s]


KeyboardInterrupt: 

In [ ]:
generate_order_variance_plot(model, 'mnist', grayscale_preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

In [ ]:
generate_order_variance_plot(model, 'eurosat', preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

In [ ]:
generate_order_variance_plot(model, 'cars', preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

In [ ]:
generate_order_variance_plot(model, 'food101', preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

In [ ]:
generate_order_variance_plot(model, 'cifar10', preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

In [ ]:
generate_order_variance_plot(model, 'cifar100', preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

In [ ]:
generate_order_variance_plot(model, 'cub', preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

In [ ]:
generate_order_variance_plot(model, 'gtsrb', preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

In [ ]:
generate_order_variance_plot(model, 'flowers', preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

In [ ]:
generate_order_variance_plot(model, 'aircraft', preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

In [27]:
ds_names = ['dtd', 'mnist', 'eurosat', 'cars', 'food101', 'cifar10',
           'cifar100', 'cub', 'gtsrb', 'flowers', 'aircraft']

In [30]:
for ds_name in ds_names:
    for net in [model, cc_model]:
        print(ds_name)
        print(test_on_dataset(net, ds_name,
                              grayscale_preprocess if ds_name=='mnist' else preprocess))

dtd


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 59/59 [00:05<00:00, 11.27it/s]


0.19414893617021275
dtd


  0%|                                                                                                                                                     | 0/313 [00:00<?, ?it/s]

0.1803191489361702
mnist


...

0.3018376971865344
aircraft
NOTE PROPER METRIC IS MEAN PER CLASS


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 105/105 [00:29<00:00,  3.54it/s]


0.0195019501950195
aircraft
NOTE PROPER METRIC IS MEAN PER CLASS


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 105/105 [00:20<00:00,  5.09it/s]


0.0306030603060306
